In [6]:
from flask import Flask, request, jsonify
import numpy as np
import joblib

app = Flask(__name__)

# Load saved models, scalers, and encoders
rf_tshirt_model = joblib.load("litfit/le_jeans.pkl")
rf_jeans_model = joblib.load("litfit/rf_jeans_model.pkl")
scaler_ts = joblib.load("litfit/scaler_ts.pkl")
scaler_je = joblib.load("litfit/scaler_je.pkl")
le_tshirt = joblib.load("litfit/le_tshirt.pkl")
le_jeans = joblib.load("litfit/le_jeans.pkl")

@app.route('/predict', methods=['POST'])
def predict():
    # Expect JSON payload with keys: "ChestCMS", "WaistCMS", "HeightCMS", "WeightKGS"
    data = request.get_json(force=True)

    try:
        chest = float(data.get("ChestCMS"))
        waist = float(data.get("WaistCMS"))
        height = float(data.get("HeightCMS"))
        weight = float(data.get("WeightKGS"))
    except Exception as e:
        return jsonify({"error": "Invalid input. Please provide numeric values for ChestCMS, WaistCMS, HeightCMS, and WeightKGS."}), 400

    # Create an input array in the same order as used during training
    input_array = np.array([[chest, waist, height, weight]], dtype=np.float32)

    # Scale the input using the saved scalers (separately for T-Shirt and Jeans models)
    X_ts_scaled = scaler_ts.transform(input_array)
    X_je_scaled = scaler_je.transform(input_array)

    # Predict using the loaded RandomForest models
    tshirt_pred_enc = rf_tshirt_model.predict(X_ts_scaled)
    jeans_pred_enc = rf_jeans_model.predict(X_je_scaled)

    # Convert numeric predictions back to their original labels
    tshirt_pred = le_tshirt.inverse_transform([int(tshirt_pred_enc)])
    jeans_pred = le_jeans.inverse_transform([int(jeans_pred_enc)])

    # Prepare JSON output
    output = {
        "body_measurements": {
            "ChestCMS": chest,
            "WaistCMS": waist,
            "HeightCMS": height,
            "WeightKGS": weight
        },
        "size_recommendation": {
            "T-Shirt": tshirt_pred[0],
            "Pants": jeans_pred[0]
        }
    }

    return jsonify(output)

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [22/Feb/2025 23:41:30] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Feb/2025 23:41:30] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Feb/2025 23:41:31] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Feb/2025 23:41:55] "GET /predict HTTP/1.1" 405 -
127.0.0.1 - - [22/Feb/2025 23:43:34] "GET /predict HTTP/1.1" 405 -
C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
127.0.0.1 - - [22/Feb/2025 23:45:36] "POST /predict HTTP/1.1" 500 -
Traceback (most recent call last):
  File "C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\flask\app.py", li

In [2]:
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS
import numpy as np
import pandas as pd
import joblib

app = Flask(__name__)
CORS(app)  # Enable CORS for all routes

# Load saved models, scalers, and label encoders.
rf_tshirt_model = joblib.load("rf_tshirt_model.pkl")
rf_jeans_model = joblib.load("rf_jeans_model.pkl")
scaler_ts = joblib.load("scaler_ts.pkl")
scaler_je = joblib.load("scaler_je.pkl")
le_tshirt = joblib.load("le_tshirt.pkl")
le_jeans = joblib.load("le_jeans.pkl")

# A simple home endpoint to verify the API is running.
@app.route('/', methods=['GET'])
def home():
    return "API is running. Use POST /predict to get predictions."

@app.route('/predict', methods=['POST'])
def predict():
    # Expect a JSON payload with keys: "ChestCMS", "WaistCMS", "HeightCMS", "WeightKGS"
    data = request.get_json(force=True)

    try:
        chest = float(data["ChestCMS"])
        waist = float(data["WaistCMS"])
        height = float(data["HeightCMS"])
        weight = float(data["WeightKGS"])
    except Exception as e:
        return jsonify({
            "error": "Invalid input. Provide numeric values for ChestCMS, WaistCMS, HeightCMS, and WeightKGS."
        }), 400

    # Convert input to DataFrame with the same column names as used during training.
    input_df = pd.DataFrame([[chest, waist, height, weight]],
                            columns=["ChestCMS", "WaistCMS", "HeightCMS", "WeightKGS"])

    # Scale the input using the saved scalers.
    X_ts_scaled = scaler_ts.transform(input_df)
    X_je_scaled = scaler_je.transform(input_df)

    # Predict using the loaded RandomForest models.
    tshirt_pred_enc = rf_tshirt_model.predict(X_ts_scaled)
    jeans_pred_enc = rf_jeans_model.predict(X_je_scaled)

    # Convert numeric predictions back to original size labels.
    tshirt_size = le_tshirt.inverse_transform([int(tshirt_pred_enc[0])])[0]
    jeans_size = le_jeans.inverse_transform([int(jeans_pred_enc[0])])[0]

    output = {
        "body_measurements": {
            "ChestCMS": chest,
            "WaistCMS": waist,
            "HeightCMS": height,
            "WeightKGS": weight
        },
        "size_recommendation": {
            "T-Shirt": tshirt_size,
            "Jeans": jeans_size
        }
    }

    return jsonify(output)

if __name__ == '__main__':
    # Run the Flask app with use_reloader=False to avoid issues in some environments.
    app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: T

 * Serving Flask app '__main__'
 * Debug mode: on


C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\sonal\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.1 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
